#### creating database and table


In [2]:
import pymysql

# Database connection details
DB_HOST = "localhost"      # Change if using a remote server
DB_USER = "root"           # Replace with your MySQL username
DB_PASSWORD = "root"   # Replace with your MySQL password
DB_NAME = "eco_friendly"   # Database name

# Connect to MySQL
connection = pymysql.connect(
    host=DB_HOST,
    user=DB_USER,
    password=DB_PASSWORD
)

cursor = connection.cursor()

# Create database if not exists
cursor.execute(f"CREATE DATABASE IF NOT EXISTS {DB_NAME};")
print(f"Database '{DB_NAME}' created or already exists.")

# Connect to the newly created database
connection.select_db(DB_NAME)

# Confirm connection
print(f"Connected to database: {DB_NAME}")

# Close the connection
cursor.close()
connection.close()


Database 'eco_friendly' created or already exists.
Connected to database: eco_friendly


### inserting data into database

In [7]:
import pymysql
import pandas as pd

# Database connection details
DB_HOST = "localhost"
DB_USER = "root"
DB_PASSWORD = "root"
DB_NAME = "eco_friendly"
TABLE_NAME = "product"

# Connect to MySQL
connection = pymysql.connect(
    host=DB_HOST,
    user=DB_USER,
    password=DB_PASSWORD,
    database=DB_NAME
)
cursor = connection.cursor()

# Drop the table if it exists (Ensure clean schema)
cursor.execute(f"DROP TABLE IF EXISTS {TABLE_NAME}")

# ✅ Create table with user_id column
create_table_query = f"""
CREATE TABLE {TABLE_NAME} (
    product_id INT AUTO_INCREMENT PRIMARY KEY,
    user_id INT,  -- Now explicitly defining user_id
    product_name VARCHAR(255),
    category VARCHAR(100),
    material VARCHAR(255),
    price VARCHAR(50),
    description TEXT,
    brand VARCHAR(100),
    availability VARCHAR(50),
    ratings FLOAT
);
"""
cursor.execute(create_table_query)
connection.commit()

# Load data from Excel
file_path = "EcoFriendly_Products.xlsx"  # Ensure correct path
df = pd.read_excel(file_path)

# ✅ Ensure 'User_ID' column exists in DataFrame
if "User_ID" not in df.columns:
    raise ValueError("❌ Error: 'User_ID' column is missing in Excel file.")

# ✅ Insert data into MySQL
insert_query = f"""
INSERT INTO {TABLE_NAME} (user_id, product_name, category, material, price, description, brand, availability, ratings)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s);
"""

for _, row in df.iterrows():
    cursor.execute(insert_query, (
        int(row["User_ID"]),  
        row["Product Name"], 
        row["Category"], 
        row["Material"], 
        row["Price"],
        row["Description"], 
        row["Brand"], 
        row["Availability"], 
        float(row["Ratings"]) if not pd.isna(row["Ratings"]) else 0.0  
    ))

connection.commit()
print("✅ Data successfully inserted into the database with original user IDs!")

# Close the connection
cursor.close()
connection.close()


ValueError: ❌ Error: 'User_ID' column is missing in Excel file.

In [11]:
import pymysql
import pandas as pd

# Database connection details
DB_HOST = "localhost"
DB_USER = "root"
DB_PASSWORD = "root"
DB_NAME = "eco_friendly"
TABLE_NAME = "products"  # Ensure consistency in table name

# Connect to MySQL
connection = pymysql.connect(
    host=DB_HOST,
    user=DB_USER,
    password=DB_PASSWORD,
    database=DB_NAME
)
cursor = connection.cursor()

# Drop the table if it exists (Ensure clean schema)
cursor.execute(f"DROP TABLE IF EXISTS {TABLE_NAME}")

# ✅ Create table with user_id column
create_table_query = f"""
CREATE TABLE {TABLE_NAME} (
    product_id INT AUTO_INCREMENT PRIMARY KEY,
    user_id INT,  
    product_name VARCHAR(255),
    category VARCHAR(100),
    material VARCHAR(255),
    price VARCHAR(50),
    description TEXT,
    brand VARCHAR(100),
    availability VARCHAR(50),
    ratings FLOAT
);
"""
cursor.execute(create_table_query)
connection.commit()

# Load data from Excel
file_path = "EcoFriendly_Products.xlsx"  # Ensure correct path
df = pd.read_excel(file_path)

# ✅ Display actual column names for debugging
print("🔍 Columns in Excel file:", df.columns.tolist())

# ✅ Dynamically find 'User_ID' column (case-insensitive, including 'User-ID')
user_id_col = None
for col in df.columns:
    if col.strip().lower() in ["user_id", "userid", "user id", "user-id"]:
        user_id_col = col
        break

if not user_id_col:
    raise ValueError("❌ Error: Could not find a 'User_ID' column in the Excel file.")

# ✅ Insert data into MySQL
insert_query = f"""
INSERT INTO {TABLE_NAME} (user_id, product_name, category, material, price, description, brand, availability, ratings)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s);
"""

for _, row in df.iterrows():
    cursor.execute(insert_query, (
        int(row[user_id_col]) if pd.notna(row[user_id_col]) else None,  # Use dynamically found column
        row["Product Name"] if pd.notna(row["Product Name"]) else "Unknown",
        row["Category"] if pd.notna(row["Category"]) else "Unknown",
        row["Material"] if pd.notna(row["Material"]) else "Unknown",
        row["Price"] if pd.notna(row["Price"]) else "0",
        row["Description"] if pd.notna(row["Description"]) else "No description",
        row["Brand"] if pd.notna(row["Brand"]) else "Unknown",
        row["Availability"] if pd.notna(row["Availability"]) else "Unknown",
        float(row["Ratings"]) if pd.notna(row["Ratings"]) else 0.0  # Handle NaN in Ratings
    ))

connection.commit()
print("✅ Data successfully inserted into the database with original user IDs!")

# Close the connection
cursor.close()
connection.close()


🔍 Columns in Excel file: ['Product ID', 'Product Name', 'Category', 'Material', 'Price', 'Description', 'Brand', 'Availability', 'User-ID', 'Ratings']
✅ Data successfully inserted into the database with original user IDs!


### extracting data from database

In [14]:
import pymysql
import pandas as pd

# Database connection details
HOST = "localhost"
USER = "root"
PASSWORD = "root"
DATABASE = "eco_friendly"

try:
    # Connect to MySQL database using pymysql
    connection = pymysql.connect(
        host=HOST,
        user=USER,
        password=PASSWORD,
        database=DATABASE
    )

    print("✅ Connected to MySQL Database")

    # Query to fetch all data from the Products table
    query = "SELECT * FROM products"  # Ensure table name is correct

    # Load data into a Pandas DataFrame
    df = pd.read_sql(query, connection)

    print("✅ Data successfully extracted from MySQL")
    print(df.head())  # Display first 5 rows

except pymysql.MySQLError as e:
    print(f"❌ Error: {e}")

finally:
    if connection:
        connection.close()
        print("✅ MySQL connection is closed")



✅ Connected to MySQL Database
✅ Data successfully extracted from MySQL
   product_id  user_id                 product_name    category  \
0           1      100            Bamboo Toothbrush  Toothbrush   
1           2      101  Recycled Plastic Toothbrush  Toothbrush   
2           3      102        Cornstarch Toothbrush  Toothbrush   
3           4      103          Silicone Toothbrush  Toothbrush   
4           5      104  Charcoal-infused Toothbrush  Toothbrush   

           material  price                                        description  \
0            Bamboo  45.67  Eco-friendly Toothbrush made from Bamboo. Sust...   
1  Recycled Plastic  24.41  Eco-friendly Toothbrush made from Recycled Pla...   
2        Cornstarch  16.19  Eco-friendly Toothbrush made from Cornstarch. ...   
3          Silicone  48.35  Eco-friendly Toothbrush made from Silicone. Su...   
4  Charcoal-infused  15.17  Eco-friendly Toothbrush made from Charcoal-inf...   

      brand  availability  ratings  
0 

C:\Users\Dell\AppData\Local\Temp\ipykernel_11336\4025013269.py:25: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, connection)


###  Content based filtering using cosine similarity

In [16]:

import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity



# Step 2: Get Unique Categories & Ask User to Select One
unique_categories = df['category'].unique()
print("Available Categories:", unique_categories)

selected_category = input("Enter the category you want recommendations from: ").strip()

# Step 3: Filter Products by Selected Category
df_filtered = df[df['category'] == selected_category].reset_index(drop=True)

# Step 4: Combine 'category', 'material', and 'description' into a single feature
df_filtered['combined_features'] = df_filtered['category'] + " " + df_filtered['material'] + " " + df_filtered['description']

# Step 5: Convert text data into numerical form (TF-IDF Vectorization)
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df_filtered['combined_features'])

# Step 6: Compute Similarity Matrix
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Step 7: Create Recommendation Function
def recommend_products(product_name, num_recommendations=5):
    # Check if the product exists in the selected category
    if product_name not in df_filtered['product_name'].values:
        print("Product not found in the selected category.")
        return
    
    # Find index of the product
    idx = df_filtered[df_filtered['product_name'] == product_name].index[0]
    
    # Get similarity scores for all products in the selected category
    sim_scores = list(enumerate(cosine_sim[idx]))
    
    # Sort products by similarity score (highest first)
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # Get top N similar products (excluding itself)
    sim_scores = sim_scores[1:num_recommendations+1]
    
    # Get recommended product indices
    product_indices = [i[0] for i in sim_scores]
    
    # Return top recommended products
    return df_filtered[['product_name', 'category', 'material']].iloc[product_indices]

# Step 8: Ask User for a Product Name in the Selected Category
print("\nAvailable Products in Category:", selected_category)
print(df_filtered['product_name'].tolist())

product_to_search = input("Enter the product name for recommendations: ").strip()

# Step 9: Get Recommendations
recommendations = recommend_products(product_to_search, 5)

if recommendations is not None:
    print("\nRecommended Products:")
    print(recommendations)

Available Categories: ['Toothbrush' 'Bag' 'Water Bottle' 'Clothing' 'Cutlery' 'Notebook' 'Shoes'
 'Toys' 'Straws' 'Phone Cases']


Enter the category you want recommendations from:  Bag



Available Products in Category: Bag
['Jute Bag', 'Hemp Bag', 'Organic Cotton Bag', 'Recycled Plastic Bag', 'Cork Bag', 'Canvas Bag', 'Bamboo Fiber Bag', 'Paper Bag', 'Upcycled Denim Bag', 'Tyvek Bag']


Enter the product name for recommendations:  Canvas Bag



Recommended Products:
  product_name category material
0     Jute Bag      Bag     Jute
1     Hemp Bag      Bag     Hemp
4     Cork Bag      Bag     Cork
7    Paper Bag      Bag    Paper
9    Tyvek Bag      Bag    Tyvek


###  Content based filtering using euclidean_distances

In [18]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import euclidean_distances

# Step 1: Load Data (Assuming df is already loaded)
# Step 2: Get Unique Categories & Ask User to Select One
unique_categories = df['category'].unique()
print("Available Categories:", unique_categories)

selected_category = input("Enter the category you want recommendations from: ").strip()

# Step 3: Filter Products by Selected Category
df_filtered = df[df['category'] == selected_category].reset_index(drop=True)

# Step 4: Combine 'category', 'material', and 'description' into a single feature
df_filtered['combined_features'] = df_filtered['category'] + " " + df_filtered['material'] + " " + df_filtered['description']

# Step 5: Convert text data into numerical form (TF-IDF Vectorization)
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df_filtered['combined_features'])

# Step 6: Compute Similarity Matrix using Euclidean Distance
euclidean_sim = euclidean_distances(tfidf_matrix, tfidf_matrix)

# Step 7: Create Recommendation Function
def recommend_products(product_name, num_recommendations=5):
    if product_name not in df_filtered['product_name'].values:
        print("❌ Product not found in the selected category.")
        return
    
    # Find index of the product
    idx = df_filtered[df_filtered['product_name'] == product_name].index[0]
    
    # Get Euclidean distances for all products in the selected category
    dist_scores = list(enumerate(euclidean_sim[idx]))
    
    # Sort products by distance (lowest first because Euclidean distance is dissimilarity)
    dist_scores = sorted(dist_scores, key=lambda x: x[1])
    
    # Get top N similar products (excluding itself)
    dist_scores = dist_scores[1:num_recommendations+1]
    
    # Get recommended product indices
    product_indices = [i[0] for i in dist_scores]
    
    # Return top recommended products
    return df_filtered[['product_name', 'category', 'material']].iloc[product_indices]

# Step 8: Ask User for a Product Name in the Selected Category
print("\nAvailable Products in Category:", selected_category)
print(df_filtered['product_name'].tolist())

product_to_search = input("Enter the product name for recommendations: ").strip()

# Step 9: Get Recommendations
recommendations = recommend_products(product_to_search, 5)

if recommendations is not None:
    print("\nRecommended Products:")
    print(recommendations)


Available Categories: ['Toothbrush' 'Bag' 'Water Bottle' 'Clothing' 'Cutlery' 'Notebook' 'Shoes'
 'Toys' 'Straws' 'Phone Cases']


Enter the category you want recommendations from:  Cutlery



Available Products in Category: Cutlery
['Bamboo Cutlery', 'Coconut Shell Cutlery', 'Recycled Stainless Steel Cutlery', 'Wheat Straw Cutlery', 'Cornstarch Cutlery', 'Wood Cutlery', 'Silicone Cutlery', 'Edible Cutlery Cutlery', 'Paper-based Cutlery', 'Rice Husk Cutlery']


Enter the product name for recommendations:  Wheat Straw Cutlery



Recommended Products:
             product_name category        material
7  Edible Cutlery Cutlery  Cutlery  Edible Cutlery
0          Bamboo Cutlery  Cutlery          Bamboo
4      Cornstarch Cutlery  Cutlery      Cornstarch
5            Wood Cutlery  Cutlery            Wood
6        Silicone Cutlery  Cutlery        Silicone


## Collaborative filtering

In [21]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import jaccard
import ipywidgets as widgets
from IPython.display import display, clear_output
# Step 1: User ID Dropdown (simulated)
user_id_dropdown = widgets.IntText(
    value=1,
    description="User ID:",
    disabled=False
)
# Step 2: Dropdown for Product Selection
product_dropdown = widgets.Dropdown(
    options=df['product_name'].unique(),
    description="Select Product:",
    disabled=False
)
# Step 3: Button to Trigger Recommendations
recommend_button = widgets.Button(
    description="Get Recommendations",
    button_style="primary"
)
# Output widget to display results
output = widgets.Output()
# Function to Calculate Jaccard Similarity
def calculate_jaccard_similarity(df):
    # Ensure 'price' and 'ratings' are numeric
    df[['price', 'ratings']] = df[['price', 'ratings']].apply(pd.to_numeric, errors='coerce')
    # Convert to binary (above/below median)
    binary_features = np.where(df[['price', 'ratings']] > df[['price', 'ratings']].median(), 1, 0)
    # Compute Jaccard similarity matrix
    num_products = len(df)
    sim_matrix = np.zeros((num_products, num_products))
    for i in range(num_products):
        for j in range(num_products):
            if i != j:
                sim_matrix[i][j] = 1 - jaccard(binary_features[i], binary_features[j])
    return sim_matrix
# Function to Get Recommendations
def recommend_products(user_id, product_name, num_recommendations=5):
    with output:
        clear_output()
        # Check if product exists
        if product_name not in df['product_name'].values:
            print("\n:warning: Product not found in dataset.")
            return
        # Compute Jaccard similarity
        similarity_matrix = calculate_jaccard_similarity(df)
        # Get index of selected product
        product_idx = df[df['product_name'] == product_name].index[0]
        # Find similar products
        sim_scores = list(enumerate(similarity_matrix[product_idx]))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        sim_scores = sim_scores[1:num_recommendations + 1]  # Exclude the selected product itself
        # Get recommended product indices
        product_indices = [i[0] for i in sim_scores]
        print("\n:small_blue_diamond: Recommended Products Based on:", product_name, "(Jaccard Similarity)")
        display(df.iloc[product_indices])
# Function to Handle Button Click
def on_button_click(b):
    selected_product = product_dropdown.value
    user_id = user_id_dropdown.value  # Get user_id from widget
    if selected_product:
        recommend_products(user_id, selected_product, 5)
# Link button to function
recommend_button.on_click(on_button_click)
# Display widgets
display(user_id_dropdown, product_dropdown, recommend_button, output)

IntText(value=1, description='User ID:')

Dropdown(description='Select Product:', options=('Bamboo Toothbrush', 'Recycled Plastic Toothbrush', 'Cornstar…

Button(button_style='primary', description='Get Recommendations', style=ButtonStyle())

Output()

## Hybrid fitlering

In [24]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import jaccard
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import ipywidgets as widgets
from IPython.display import display, clear_output
# User Input Widgets
user_id_dropdown = widgets.IntText(value=1, description="User ID:", disabled=False)
product_dropdown = widgets.Dropdown(options=df['product_name'].unique(), description="Select Product:", disabled=False)
recommend_button = widgets.Button(description="Get Recommendations", button_style="primary")
output = widgets.Output()
# Function to Compute Jaccard Similarity
def calculate_jaccard_similarity(df):
    df[['price', 'ratings']] = df[['price', 'ratings']].apply(pd.to_numeric, errors='coerce')
    binary_features = np.where(df[['price', 'ratings']] > df[['price', 'ratings']].median(), 1, 0)
    num_products = len(df)
    sim_matrix = np.zeros((num_products, num_products))
    for i in range(num_products):
        for j in range(num_products):
            if i != j:
                sim_matrix[i][j] = 1 - jaccard(binary_features[i], binary_features[j])
    return sim_matrix
# Function to Compute Content Similarity
def calculate_content_similarity(df):
    df['combined_features'] = df['category'] + " " + df['material'] + " " + df['description']
    tfidf = TfidfVectorizer(stop_words='english')
    tfidf_matrix = tfidf.fit_transform(df['combined_features'])
    return cosine_similarity(tfidf_matrix, tfidf_matrix)
# Hybrid Recommendation Function
def recommend_products(user_id, product_name, num_recommendations=5):
    with output:
        clear_output()
        if product_name not in df['product_name'].values:
            print(":warning: Product not found in dataset.")
            return
        jaccard_matrix = calculate_jaccard_similarity(df)
        content_matrix = calculate_content_similarity(df)
        product_idx = df[df['product_name'] == product_name].index[0]
        hybrid_scores = (jaccard_matrix[product_idx] + content_matrix[product_idx]) / 2
        sim_scores = list(enumerate(hybrid_scores))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        sim_scores = sim_scores[1:num_recommendations + 1]
        product_indices = [i[0] for i in sim_scores]
        print(":small_blue_diamond: Recommended Products Based on:", product_name)
        display(df.iloc[product_indices])
# Button Click Function
def on_button_click(b):
    selected_product = product_dropdown.value
    user_id = user_id_dropdown.value
    if selected_product:
        recommend_products(user_id, selected_product, 5)
recommend_button.on_click(on_button_click)
# Display Widgets
display(user_id_dropdown, product_dropdown, recommend_button, output)

IntText(value=1, description='User ID:')

Dropdown(description='Select Product:', options=('Bamboo Toothbrush', 'Recycled Plastic Toothbrush', 'Cornstar…

Button(button_style='primary', description='Get Recommendations', style=ButtonStyle())

Output()